## 6.1 从全连接层到卷积
在之前猫狗分类的例子中：假设我们有一个足够充分的照片数据集，数据集中是拥有标注的照片，每张照片具有百万级像素，这意味着网络的每次输入都有一百万个维度。即使将隐藏层维度降低到1000，这个全连接层也将有$10^6 \times 10^3 = 10^9$个参数。想要训练这个模型将不可实现，因为需要大量的GPU、分布式优化训练的经验和耐心。
卷积神经网络(Convolutional Neural Network，CNN)是机器学习利用自然图像中一些已知结构的创造性方法

### 6.1.1 不变性
想象一下，假设你想从一张图片中找到某个物体。合理的假设是：无论哪种方法找到这个物体，都应该和物体的位置无关。理想情况下，我们的系统应该能利用常识：猪通常不在天上飞，飞机通常不在水里游泳。但是，如果一只猪出现在图片的顶部，我们还是应该认出它来。我们可以从儿童游戏“沃尔多在哪里”中得到灵感：在这个游戏中包含了许多充斥着活动的混乱场景，而沃尔多通常潜伏在一些不太可能的位置，读者的目标就是找到她。尽管沃尔多的装扮很有特点，但是在眼花缭乱的场景中找到他也很有难度。然而沃尔多的样子并不取决于他潜藏的地方，因此我们可以使用一个“沃尔多检测器”扫描图像。该检测器将图像分割成多个区域，并为每个区域包含沃尔多的可能性打分。卷积神经网络正是将空间不变形(spatial invariance)的这一概念系统化，从而基于这个模型使用较少的参数来学习有用的表示。

<div align=center>
<img src='../../pics/6_1.jpeg' width='50%'>
</div>

现在，我们将上述想法总结一下，从而帮助我们设计适合于计算机视觉的神经网络架构：

1. 平移不变性(translation invariance)：不管检测对象出现在图像中的哪个位置，神经网络的前面几层应该对相同的图像区域具有相似的反应，即为“平移不变性”
2. 局部性(locality)：神经网络的前面几层应该只探索输入图像中的局部区域，而不过度在意图像中相隔较远的区域的关系，这就是“局部性”原则。最终，可以聚合这些局部特征，以在整个图像级别进行预测。


### 6.1.2 多层感知机的限制
首先，多层感知机的输入是二维图像$X$，其隐藏表示$H$在数学上是一个矩阵，在代码中表示为二维张量。其中$X$和$H$具有相同的形状。

使用$[X]_{i,j}和[H]_{i,j}$分别表示输入图像和隐藏表示中位置$(i,j)$处的像素。为了使每个隐藏神经元都能接收到每个输入像素的信息，我们将参数从权重矩阵替换为四阶权重张量$W$。假设$U$包含偏置参数，我们可以将全连接层形式化地表示为
$$
\begin{align*}
[H]_{i,j} &= [U]_{i,j} + \sum_k \sum_l [W]_{i,j,k,l} [X]_{k,l} \\
&= [U]_{i,j} + \sum_a \sum_b [V]_{i,j,a,b} [X]_{i+a, j+b}
\end{align*}
$$

其中，从$W$到$V$的转换只是形式上的转换，因为在这两个四阶张量的元素之间存在一一对应的关系。我们只需重新索引下标$(k,l)$，使$k=i+a、l=j+b$，由此可得$[V]_{i,j,a,b}= [W]_{i,j,i+a,j+b}$。索引$a$和$b$通过在正偏移和负偏移之间移动覆盖了整个图像。对于隐藏表示中任意给定位置$(i,j)$处的像素值$[H]_{i,j}$，可以通过在$x$中以$(i,j)$为中心对像素进行加权求和得到，加权使用的权重为$[V]_{i,j,a,b}$

**平移不变性**

现在引用上述的第一个原则：平移不变性。这意味着检测对象在输入$X$中的平移，应该仅导致隐藏表示$H$中的平移。也就是说，$V$和$U$实际上不依赖于$(i,j)$的值，即$[V]_{i,j,a,b}=[V]_{a,b}$。并且$U$是一个常数，比如$u$。因此，我们可以简化$H$定义为：
$$
[H]_{i,j} = u + \sum_a \sum_b [V]_{a,b} [X]_{i+a, j+b}
$$

这就是卷积(convolution)。我们是在使用系数$[V]_{a,b}$对位置$(i,j)$附近的像素$(i+a, j+b)$进行加权得到$[H]_{i,j}$。注意，$[V]_{a,b}$的系数比$[V]_{i,j,a,b}$少很多，因为前者不再依赖于图像中的位置。

**局部性**

现在引用上述的第二个原则：局部性。如上所述，为了收集用来训练参数$[H]_{i,j}$的相关信息，我们不应偏离到距$(i,j)$很远的地方。这意味着在$\mid a \mid > \bigtriangleup$或$\mid b \mid > \bigtriangleup$的范围之外，我们可以设置$[V]_{a, b}=0$。因此，我们可以将$[H]_{i,j}$重写为
$$
[H]_{i,j} = u + \sum_{a= -\bigtriangleup}^{\bigtriangleup} \sum_{b= -\bigtriangleup}^{\bigtriangleup} [V]_{a,b} [X]_{i+a,j+b}
$$

简而言之，上述公式就是一个卷积层(convolutional layer)，而卷积神经网络是包含卷积层的一类特殊的神经网络。在深度学习研究社区中，$V$被称为卷积核(convolution kernel)或者滤波器(filter)，或者称为卷积层的权重，通常该权重是可学习的参数。当图像处理的局部区域很小时，卷积神经网络与多层感知机的训练差异可能是巨大的：以前，MLP可能需要数十亿个参数来表示网络中的一层，而现在CNN通常只需要几百万个参数，而且不需要改变输入或隐藏表示的维数。参数大幅减少的代价是，我们的特征现在是平移不变的，并且当确定每个隐藏活性值时，每一层只包含局部的信息。以上所有的权重学习都将依赖于归纳偏置。当这种偏置与现实相符时，我们就能得到样本有效的模型，并且这些模型能很好地泛化到未知数据。但如果这偏置与现实不符，比如当图像不满足平移不变时，我们的模型可能难以拟合我们的训练数据。

### 6.1.3 卷积
在进一步讨论之前，我们先简要回顾一下为什么上面的操作被称为卷积。在数学中，两个函数(比如$f,g$: \mathbb R^d \rightarrow \mathbb R)之间的“卷积”被定义为
$$
(f * g)(x) = \int f(z)g(x-z) dz
$$

也就是说，卷积是当把一个函数“翻转”并移位$x$时，测量$f$和$g$之间的重叠。当为离散对象时，积分就变成了求和。例如：对于由索引为$\mathbb Z$的、平方和的、无限维向量集合中抽取的向量，我们得到以下定义：
$$
(f*g)(i) = \sum_a f(a) g(i-a)
$$
对于二维张量，则为$f$的索引$(a,b)$和$g$的索引$(i-a, j-b)$上的对应加和：
$$
(f * g)(i,j) = \sum_a \sum_b f(a,b) g(i-a, j-b)
$$

### 6.1.4 "沃尔多在哪里"回顾

回到上面的“沃尔多在哪里”游戏，让我们看看它到底是什么样子。卷积层根据滤波器$V$选取给定大小的窗口，并加权处理图片，如下图所示。我们的目标是学习一个模型，以便探测出在“沃尔多”最可能出现的地方。

<div align=center>
<img src='../../pics/6_1_2.jpeg' width='50%'>
</div>

**通道**

然而这种方法有一个问题：我们忽略了图像一般包含三个通道/三种原色(红、绿和蓝)。实际上，图像不是二维张量，而是一个由高度、宽度和颜色组成的三位张量，比如包含$1024 \times 1024 \times 3$个像素。前两个轴与像素的空间位置有关，而第三个轴可以看作是每个像素的多维表示。因此，我们将$X$索引为$[X]_{i,j,k}$。由此卷积相应地调整为$[V]_{a,b,c}$，而不是$[V]_{a,b}$。
此外，由于输入图像是三维的，我们的隐藏表示$H$也最好采用三维张量。换句话说，对于每一个空间位置，我们想要采用一组而不是一个隐藏表示。这样一组隐藏表示可以想象成一些互相堆叠的二维网络。因此，我们可以把隐藏表示想象为一系列具有二维张量的通道(channel)。这些通道有时也被称为特征映射(feature maps)，因为每个通道都向后续层提供一组空间化的学习特征。直观上你可以想象在靠近输入的底层，一些通道专门识别边缘，而一些通道专门识别纹理。

为了支持输入$X$和隐藏表示$H$中的多个通道，我们可以在$V$中添加第四个坐标，即$[V]_{a,b,c,d}$。综上所述，
$$
[H]_{i,j,d} = \sum_{a=- \bigtriangleup}^{\bigtriangleup} \sum_{b=- \bigtriangleup}^{\bigtriangleup} \sum_c [V]_{a,b,c,d} [X]_{i+a, j+b, c}
$$

其中隐藏表示$H$中的索引$d$表示输出通道，而随后的输出将继续以三维张量$H$作为输入进入下一个卷积层。

### 6.1.5 小结
- 图像的平移不变性使我们以相同的方式处理局部图像，而不在乎它的位置。
- 局部性意味着计算相应的隐藏表示只需一小部分局部图像像素。
- 在图像处理中，卷积层通常比全连接层需要更少的参数，但依旧获得高效用的模型
- 卷积神经网络(CNN)是一类特殊的神经网络，它可以包含多个卷积层
- 多个输入和输出通道使模型在每个空间位置获取图像的多方面特征